In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import random
import math
import matplotlib.pyplot as plt

In [2]:
x = np.random.normal(size=(500,4))
error = np.random.normal(size=(500,1))
b = np.random.uniform(size=(4,1))

### in order to avoid Perfect Separation, we need to add some noise
y = x@b + error

y = list(map(lambda x: 1 if x > 0 else 0, y))
y = np.array(y).reshape(-1,1)

In [3]:
class data_generate_process:
    def __init__(self, x, y):
        self.x = x
        self.y = y    
        
    def split(self, rate = 0.7, random_state = 1024, scale = False):
        ## Feature scaling is used to normalize the range of independent variables or features of data
        if scale:
            self.x = (self.x - np.mean(self.x))/x.std()
        
        n = len(self.y)
        np.random.seed(random_state)
        
        ##randomly spilte data into 70% train and 30% test
        index = list(range(n))
        np.random.shuffle(index)
        train = index[:int(rate*n)]
        test = index[int(rate*n):]
        
        self.train_x = self.x[train]
        self.test_x = self.x[test]
        self.train_y = self.y[train]
        self.test_y = self.y[test]
        
        return self.train_x, self.test_x, self.train_y, self.test_y
    
class model:
    def __init__(self, x, y):
        self.x = x
        self.y = y
     
    
    def bias(self, intercept):
        ## if need intercept, assign x0 as 1
        if intercept:
            n = len(self.x)
            ones = np.ones((n,1))
            return np.hstack([ones,self.x])
        return self.x
    def label(self, prob = None, threshold = 0.5):
        
        labels = list(map(lambda x: 1 if x > threshold else 0, prob))
        return np.array(labels).reshape(-1,1)
        
    def tidy(self, x, tails = 2):
        
        n, k = x.shape
        self.error = self.y - x@self.beta 
        if not self.vb.any():
            self.vb = self.error.var()*np.linalg.inv(x.T@x)
            
        self.se = np.sqrt(np.diagonal(self.vb)).reshape(-1,1)
            
        self.t = np.divide(self.beta,self.se)
        self.pval = tails * (1 - stats.norm.cdf(self.t))
        
        names = ['Coef','Std err','t','p-value']
        values = [self.beta, self.se, self.t, self.pval]
        values = np.hstack(values)
        
        self.summary = pd.DataFrame(values, columns =names)

        self.rsq = 1 - self.error.var()/self.y.var()
        self.adjrsq = self.rsq*(n -1)/(n-k-1)
        
        var = self.error.var()
        ## sum function here is useless, but just in order to get a list, rather than a list in a list
        ## in this case, it's easily to plot graph
        self.sse = sum(self.error.T@self.error)
        
        if not len(self.logl):
            logl = -n/2*np.log(2*var*math.pi) - 1/2/var*self.sse
        
#         self.logl = self.logl.tolist()
        
        self.aic = -2*(self.logl)+ 2*k 
        self.bic = -2*(self.logl)+ k*np.log(n) 
        
        names = ['r.squared','adj.rsq','df','loglikehood','aic','bic']
        values = [self.rsq,self.adjrsq,n-k-1,self.logl,self.aic,self.bic]
        glance= pd.DataFrame(columns = ['r.squared','adj.rsq','df','loglikehood','aic','bic'])
        glance.loc[0] = values
        
        self.glance = glance 
        return  self.summary
    
    def performance(self, test_y, y_pred):
        tn = 0
        fp = 0
        fn = 0
        tp = 0

        for i in range(len(test_y)):

            if y_pred[i] == test_y[i]:
                if y_pred[i] == 1:
                    tp += 1
                    continue

                tn += 1
                continue

            if y_pred[i] == 1:
                    fp += 1
                    continue 
            fn += 1

        self.tn, self.fp, self.fn, self.tp = tn, fp, fn, tp

        self.confusion = pd.DataFrame({'predict 0':[tn, fn], 'predict 1':[fp, tp]})
        
        self.accuray = (self.tn + self.tp)/len(test_y)
        
        self.precion = tp/(tp + fp)
        self.recall = tp/(tp + fn)
        self.fmeasure = (2 * self.precion * self.recall) / (self.recall + self.precion) 
    
        return {'': self.confusion,
            'Accuracy': self.accuray,
            'Recall': self.recall,
            'Precion': self.precion,
            'Fmeasure': self.fmeasure 
             }
    
    
    def sigmoid(self, x, beta):
        return 1/(1+np.exp(-x@beta))
    
    def Hession(self, x, beta):
        
        sigmoid = self.sigmoid(x, beta)
        
        w = np.diagflat(sigmoid*(1-sigmoid))
        return x.T@w@x
    
    def Jacobian(self, x, beta):
        
        sigmoid = self.sigmoid(x, beta)
        
        return x.T@(self.y- sigmoid)
        
    def LogisticRegression(self, algorithm = 'MLE', alpha = 0.001,iterations = int(1e+6), threshold = 1e-5, \
                           intercept = None, random_state = None):
        
        x = self.bias(intercept)
        m, n = x.shape
        
        if random_state is None:
             ## init beta = [1,1,1]
            beta = np.ones(n).reshape(-1,1)  
        else:
                ## random start
            beta = np.random.randn(n).reshape(-1,1)

        cost = []

        if algorithm == 'MLE':
            
            for i in range(iterations):
                update =  np.linalg.solve(self.Hession(x, beta), self.Jacobian(x, beta))   
                beta += update

                ## threshold measure if learning step is accuracy
                if np.abs(update).sum()< threshold:
                    self.beta = beta
                    self.vb = np.linalg.inv(self.Hession(x, beta))
                    self.logl = self.y.T@x@beta - sum(np.log(1+np.exp(x@beta)))
                    return self.tidy(x)
                
        if algorithm == 'GradientDescent':
            for i in range(iterations):
                update =  alpha/m*x.T@(1/(1+np.exp(-x@beta)) - self.y)
                beta -= update

                h = 1/np.exp(-x@beta)
                cost.append(sum((h - self.y).T@(h - self.y)/len(self.y)))
                if (i > 100) and abs(cost[-1] - cost[len(cost)-2]) <= 10e-5:
                    self.beta = beta
                    self.cost = cost
                    return beta
            
        return Exception('Gradient did not converge')
    

    
    def predict(self, model, x, beta = None):
        if not beta:
            beta = self.beta
            
        if model == 'LogisticRegression':
            self.prob = self.sigmoid(x, beta)
            return self.label(self.prob)
        
        if model == 'LinearRegression':
            return x@bta
        
        return Exception('Model is not Supportable')
        


train_x, test_x, train_y, test_y = data_generate_process(x, y ).split()

In [4]:
## default use MaxLikehood Estimation
logit = model(train_x, train_y)
logit.LogisticRegression()

,Coef,Std err,t,p-value
0,0.458605,0.139727,3.282139,1.030228e-03
1,1.240656,0.169700,7.310860,2.653433e-13
2,0.546233,0.135461,4.032404,5.520915e-05
3,1.237859,0.166907,7.416462,1.203482e-13


In [5]:
logit.glance

,r.squared,adj.rsq,df,loglikehood,aic,bic
0,-10.834829,-10.96045,345,[[-162.30998659562013]],[[332.61997319124026]],[[348.0517058091741]]


In [6]:
import statsmodels.api as sm
gamma_model = sm.Logit(train_y, train_x)

gamma_results = gamma_model.fit()

print(gamma_results.summary())

## same as the package 

Optimization terminated successfully.
         Current function value: 0.463743
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  350
Model:                          Logit   Df Residuals:                      346
Method:                           MLE   Df Model:                            3
Date:                Sat, 11 Apr 2020   Pseudo R-squ.:                  0.3307
Time:                        17:31:50   Log-Likelihood:                -162.31
converged:                       True   LL-Null:                       -242.51
Covariance Type:            nonrobust   LLR p-value:                 1.502e-34
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.4586      0.140      3.282      0.001       0.185       0.732
x2             1.2407      0.

In [7]:
logit.LogisticRegression(random_state = True, algorithm = 'GradientDescent')
## it's not atble but you can get more accuracy over multiple time

array([[ 0.2949109 ],
       [ 0.60422571],
       [-0.10776678],
       [ 1.39051323]])

In [8]:
logit.LogisticRegression( 'GradientDescent')

array([[0.98570343],
       [1.00825686],
       [0.98560841],
       [1.00731386]])

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
clf = LogisticRegression(random_state=0)
clf.fit(train_x, train_y.ravel())
clf.coef_

## sklearn estimates coefficients.

C:\Users\fz_ws\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


array([[0.44729587, 1.19061913, 0.53001824, 1.19109257]])

In [10]:
logit.LogisticRegression( 'GradientDescent')
y_pred = logit.predict('LogisticRegression', test_x) 
logit.performance(test_y, y_pred)

{'':    predict 0  predict 1
 0         47         19
 1         25         59,
 'Accuracy': 0.7066666666666667,
 'Recall': 0.7023809523809523,
 'Precion': 0.7564102564102564,
 'Fmeasure': 0.728395061728395}

In [11]:
logit.LogisticRegression()
y_pred = logit.predict('LogisticRegression', test_x) 
logit.performance(test_y, y_pred)

## slice different from MLE and gradient descent

{'':    predict 0  predict 1
 0         46         20
 1         23         61,
 'Accuracy': 0.7133333333333334,
 'Recall': 0.7261904761904762,
 'Precion': 0.7530864197530864,
 'Fmeasure': 0.7393939393939394}

In [12]:
print( classification_report(test_y, clf.predict(test_x)))
print('confusion_matrix \n',confusion_matrix(test_y, clf.predict(test_x)))
## sklearn is not as good as expected in this case, but it really depedences on the data

              precision    recall  f1-score   support

           0       0.67      0.67      0.67        66
           1       0.74      0.74      0.74        84

   micro avg       0.71      0.71      0.71       150
   macro avg       0.70      0.70      0.70       150
weighted avg       0.71      0.71      0.71       150

confusion_matrix 
 [[44 22]
 [22 62]]
